# Optimization Problem

In [1]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
%matplotlib inline

In [3]:
sns.set_theme(context='paper', palette='colorblind', style='whitegrid')

## Constrained Optimization

### Linear Constraints w/o Objective Function

In [4]:
from rsnn.optim.irls import solve_lp

/Users/haguettaz/Documents/RSNN/src/rsnn/optim/irls.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [5]:
N, K = 10, 5
xb = 1

x = torch.empty(K).uniform_(-xb,xb)
A = torch.empty(N,K).normal_()
b = A@x + torch.rand(N)

In [ ]:
solve_lp(A, b, xb, device="mps")

  0%|                                                                                                                                                                                                                                                | 0/1000 [00:00<?, ?it/s]

### Linear Constraints with L2 Regularization

In [7]:
from rsnn.optim.irls import solve_lp_l2

ImportError: cannot import name 'solve_lp_l2' from 'rsnn.optim.irls' (/Users/haguettaz/Documents/RSNN/src/rsnn/optim/irls.py)

In [ ]:
def solve_lp_l2_gurobi(A, b, xb):
    _, K = A.shape
    
    model_gp = gp.Model()
    model_gp.setParam("OutputFlag", 0)
    x_gp = model_gp.addMVar(shape=K, lb=-xb, ub=xb)
    model_gp.addConstr(A @ x_gp <= b)

    lp_norm = model_gp.addVar()
    model_gp.addConstr(lp_norm == gp.norm(x_gp, 2))
    objective = lp_norm
    model_gp.setObjective(objective, gp.GRB.MINIMIZE)
    
    model_gp.optimize()

    x = x_gp.X
    
    summary = {
        "status": "optimal" if model_gp.status is gp.GRB.OPTIMAL else "infeasible", 
        "constraints": np.all(A@x <= b), 
        "bounds": np.all(np.abs(x) <= xb), 
        "l2": np.mean(np.square(x)), 
        "x":x
    }
    
    model_gp.dispose()
    
    return summary

In [ ]:
N, K = 10, 5
xb = 1

x = np.random.uniform(-xb,xb,K)
A = np.random.normal(size=(N,K))
b = A@x + np.random.uniform(0,2,N)

In [ ]:
solve_lp_l2(A, b, xb, atol=1e-12)

In [ ]:
solve_lp_l2_gurobi(A, b, xb)

### Linear Constraints with L1 Regularization

In [ ]:
from rsnn.optim.irls import solve_lp_l1

In [ ]:
def solve_lp_l1_gurobi(A, b, xb):
    _, K = A.shape
    
    model_gp = gp.Model()
    model_gp.setParam("OutputFlag", 0)
    x_gp = model_gp.addMVar(shape=K, lb=-xb, ub=xb)
    model_gp.addConstr(A @ x_gp <= b)

    lp_norm = model_gp.addVar()
    model_gp.addConstr(lp_norm == gp.norm(x_gp, 1))
    objective = lp_norm
    model_gp.setObjective(objective, gp.GRB.MINIMIZE)
    
    model_gp.optimize()

    x = x_gp.X
    
    summary = {
        "status": "optimal" if model_gp.status is gp.GRB.OPTIMAL else "infeasible", 
        "constraints": np.all(A@x <= b), 
        "bounds": np.all(np.abs(x) <= xb), 
        "l1": np.mean(np.abs(x)), 
        "x":x
    }
    
    model_gp.dispose()
    
    return summary

In [ ]:
N, K = 10, 5
xb = 1

x = np.random.uniform(-xb,xb,K)
A = np.random.normal(size=(N,K))
b = A@x + np.random.uniform(0,2,N)

In [ ]:
solve_lp_l1(A, b, xb, atol=1e-12)

In [ ]:
solve_lp_l1_gurobi(A, b, xb)

### Linear Constraints with Quantized Variables

In [ ]:
from rsnn.optim.irls import solve_lp_q
from rsnn.optim.utils import all_close_to_one_of

In [ ]:
def solve_lp_q_gurobi(A, b, xb, xlvl):
    _, K = A.shape
    
    model_gp = gp.Model()
    model_gp.setParam("OutputFlag", 0)
    x_gp = model_gp.addMVar(shape=K, lb=-xb, ub=xb, vtype=gp.GRB.INTEGER)
    model_gp.addConstr(A @ x_gp <= b)
    
    model_gp.optimize()

    x = x_gp.X
    
    summary = {
        "status": "optimal" if model_gp.status is gp.GRB.OPTIMAL else "infeasible", 
        "constraints": np.all(A@x <= b), 
        "quantization": all_close_to_one_of(x, np.linspace(-xb,xb,xlvl)), 
        "x":x
    }
    
    model_gp.dispose()
    
    return summary

In [ ]:
# should we scale the penalty strength with the number of constraints and variables?
N, K = 10, 50
xb, xlvl = 1, 3

x = np.random.randint(-xb,xb+1,K)
A = np.random.normal(size=(N,K))
b = A@x + np.random.uniform(0,1,N)

In [ ]:
solve_lp_q(A, b, xb, xlvl, gamma_y=10, max_iter=10000, atol=1e-12, rtol=1e-9)

In [ ]:
solve_lp_q_gurobi(A, b, xb, xlvl)